### 使用 VQLS 算法求解

In [1]:
# 导入必要的库
import numpy as np

import pennylane as qml

In [2]:
# 定义矩阵 A 和向量 b
A_matrix = np.array([[2, 5, -13],
              [1, -3, 1],
              [-5, 6, 8]]).astype(float)

b_vec = np.array([1000, 0, -600]).astype(float)


x_vec = np.array([1200, 500, 300]).astype(float)

# 将矩阵 A 变成一个厄尔米特矩阵
H_matrix = np.block([[np.zeros((3, 3)), A_matrix], [A_matrix.T, np.zeros((3, 3))]])

# 将 H_matrix 变为 8×8，方便用 pauli 算子的线性组合来表示
H_matrix_2 = np.zeros((8, 8))

for i in range(H_matrix.shape[0]):
    for j in range(H_matrix.shape[0]):
        H_matrix_2[i,j] = H_matrix[i,j]
        
        
# 对厄密矩阵进行 pauli 分解
# 使用 qml.pauli_decompose 进行 Pauli 分解，使用 3 个 qubit
H_decomposition = qml.pauli_decompose(H_matrix_2, wire_order=[0, 1, 2])

coeffs = H_decomposition.coeffs
ops = H_decomposition.ops


# 相对应的 b 和 x  也要进行维度拓展，以适应 H_matrixs
b_vec_2 = np.zeros((8))
b_vec_2[:3] = b_vec
b_vec_2 = b_vec_2.reshape((-1,1))

x_vec_2 = np.zeros((8))
x_vec_2[3:6] = x_vec
x_vec_2 = x_vec_2.reshape((-1,1))

# 查看拓展的线性方程组是否正确
print(np.all(H_decomposition.sparse_matrix().toarray().real @ x_vec_2 == b_vec_2))


True


In [3]:
# x_vec_2， b_vec_2 归一化成量子态

# 计算范数
b_norm = np.linalg.norm(b_vec_2)
# 归一化向量，转回行向量，方便后续计算
b_normalized = (b_vec_2 / b_norm).reshape(-1)

print("原向量: ", b_vec_2.reshape(-1))
print("归一化后的向量: ", b_normalized)

原向量:  [1000.    0. -600.    0.    0.    0.    0.    0.]
归一化后的向量:  [ 0.85749293  0.         -0.51449576  0.          0.          0.
  0.          0.        ]


In [4]:
x_norm = np.linalg.norm(x_vec_2)
x_normalized = (x_vec_2 / x_norm).reshape(-1)
print("原向量: ", x_vec_2.reshape(-1))
print("归一化后的向量: ", x_normalized)

原向量:  [   0.    0.    0. 1200.  500.  300.    0.    0.]
归一化后的向量:  [0.         0.         0.         0.89943803 0.37476584 0.22485951
 0.         0.        ]


In [11]:
# 从量子态 |x> 恢复成解 x，不需要 ||x||
b_norm * x_normalized/(np.sqrt(x_normalized.reshape(1,-1) 
@ H_matrix_2.T.conj() @ H_matrix_2 @ x_normalized.reshape(-1,1)))

array([[   0.,    0.,    0., 1200.,  500.,  300.,    0.,    0.]])